In [1]:
from astropy.cosmology import FlatLambdaCDM
from time import clock
from os import path
import glob, os
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import math
import ppxf as ppxf_package
from ppxf.ppxf import ppxf
import ppxf.ppxf_util as util
import ppxf.miles_util as lib
from astropy.table import Table, Column

In [2]:
def flux2L(flux,z):
    """Transfer flux to luminoity assuming a flat Universe"""
    cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
    DL = cosmo.luminosity_distance(z).value*10**6*3.08*10**18 # unit cm
    L = flux*1.e-17*4.*np.pi*DL**2   #erg/s/A
    return L

def L_conti(wave,flux,z):
    return np.asarray([np.log10(5100*flux2L(flux[np.where( abs(wave - 5100.) < 5.,True,False)].mean(),z) )])

In [3]:
tie_balmer=False
limit_doublets=False
ppxf_dir = path.dirname(path.realpath(ppxf_package.__file__))

mass_frac_all = np.array([])
ml_all = np.array([])
L_all = np.array([])
zbin4_syst_vel = []

In [4]:
for i in zip(glob.glob('/Users/ANguyen\MyWorks\zbin4_specs_sv\spec-*.fits')):
    try:
        #print (np.shape(i))
        hdu = fits.open(i[0])
        t = hdu[1].data
        wv = 10**t['loglam']
        redshift = float(hdu[2].data['z'])
        #print ('redshift:',np.round(redshift,4))

        wave1 = 10**t['loglam']/(1.+ redshift)
        flux1 = t['flux']
        err1 = 1./np.sqrt(t['ivar'])
        z = 0.
        mask = np.where( (wave1 > 3541.) & (wave1 < 7409.) & (err1 <100.), True, False)

        flux = flux1[mask]
        galaxy = flux/np.median(flux)   # Normalize spectrum to avoid numerical issues
        factor1=np.median(flux)
        #print 'flux normalized by:', np.median(flux)
        wave = wave1[mask]
        err = err1[mask]
        wave *= np.median(util.vac_to_air(wave)/wave)
        noise = np.full_like(galaxy, 0.01635)  # Assume constant noise per pixel here
        c = 299792.458  # speed of light in km/s
        velscale = c*np.log(wave[1]/wave[0])  # eq.(8) of Cappellari (2017)
        FWHM_gal = 2.76  # SDSS has an approximate instrumental resolution FWHM of 2.76A.
        pathname = ppxf_dir + '/miles_models/Mun1.30*.fits'
        miles = lib.miles(pathname, velscale, FWHM_gal)
        reg_dim = miles.templates.shape[1:]
        stars_templates = miles.templates.reshape(miles.templates.shape[0], -1)

        regul_err = 0.01  # Desired regularization error

        lam_range_gal = np.array([np.min(wave), np.max(wave)])/(1. + z)
        gas_templates, gas_names, line_wave = util.emission_lines(
            miles.log_lam_temp, lam_range_gal, FWHM_gal,
            tie_balmer=tie_balmer, limit_doublets=limit_doublets)
        templates = np.column_stack([stars_templates, gas_templates])

        c = 299792.458
        dv = c*(miles.log_lam_temp[0] - np.log(wave[0]))  # eq.(8) of Cappellari (2017)
        vel = c*np.log(1 + z)   # eq.(8) of Cappellari (2017)
        start = [vel, 180.]  # (km/s), starting guess for [V, sigma]

        n_temps = stars_templates.shape[1]
        n_forbidden = np.sum(["[" in a for a in gas_names])  # forbidden lines contain "[*]"
        n_balmer = len(gas_names) - n_forbidden
        component = [0]*n_temps + [1]*n_balmer + [2]*n_forbidden
        gas_component = np.array(component) > 0  # gas_component=True for gas templates
        moments = [4, 4, 4]

        start = [start, start, start]

        gas_reddening = 0 if tie_balmer else None
        t = clock()

        len_galaxy = len(galaxy)
        len_template = len(templates)
        difference = len_galaxy - len_template

        if len_galaxy > len_template:
            galaxy = galaxy[:len(galaxy)-difference]
            noise = noise[:len(noise)-difference]
            wave = noise[:len(wave)-difference]

        elif len_galaxy == len_template:
            galaxy = galaxy[:len(galaxy)-1]
            noise = noise[:len(noise)-1]
            wave = noise[:len(wave)-1]
        else:
            galaxy = galaxy
            noise = noise

        pp = ppxf(templates, galaxy, noise, velscale, start,
                  plot=False, moments=moments, degree=-1, mdegree=10, vsyst=dv,
                  lam=wave, clean=False, regul=1./regul_err, reg_dim=reg_dim,
                  component=component, gas_component=gas_component,
                  gas_names=gas_names, gas_reddening=gas_reddening)

        syst_vel = c*np.log(1 + redshift) + pp.sol[1][0]

        print ('syst_vel',syst_vel)

        zbin4_syst_vel.append(syst_vel)
    except:
        zbin4_syst_vel.append(0)
    
    #print('Desired Delta Chi^2: %.4g' % np.sqrt(2*galaxy.size))
    #print('Current Delta Chi^2: %.4g' % ((pp.chi2 - 1)*galaxy.size))

      

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in true_divide
  if sys.path[0] == '':


Emission lines included in gas templates:
['Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[OII]3726' '[OII]3729' '[SII]6716'
 '[SII]6731' '[OIII]5007_d' '[OI]6300_d' '[NII]6583_d']


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -47       108     0.020     0.019
 comp.  1:      -209       228     0.295     0.247
 comp.  2:      -196       150     0.300     0.300
chi2/DOF: 6187; degree = -1; mdegree = 10
method = capfit; Jac calls: 9; Func calls: 212; Status: 2
linear_method = lsqbox; Nonzero Templates (>0.1%):  18 / 161
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1        Hdelta      0.629     0.064    -209   228
Comp:  1        Hgamma      1.619     0.063    -209   228
Comp:  1         Hbeta      3.126     0.063    -209   228
Comp:  1        Halpha      26.06     0.062    -209   228
Comp:  2     [OII]3726      1.598     0.082    -196   150
Comp:  2     [OII]3729      6.024     0.082    -196   150
Comp:  2     [SII]6716      3.515     0.053    -196   150
Comp:  2     [SII]6731      16.09     0.053    -196   15

Emission lines included in gas templates:
['Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[OII]3726' '[OII]3729' '[SII]6716'
 '[SII]6731' '[OIII]5007_d' '[OI]6300_d' '[NII]6583_d']
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       688       437    -0.021     0.049
 comp.  1:       312       283    -0.236    -0.188
 comp.  2:       197       304    -0.016     0.060
chi2/DOF: 476.4; degree = -1; mdegree = 10
method = capfit; Jac calls: 16; Func calls: 375; Status: 2
linear_method = lsqbox; Nonzero Templates (>0.1%):  10 / 161
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1        Hdelta      3.061     0.055     312   283
Comp:  1        Hgamma      8.364     0.054     312   283
Comp:  1         Hbeta      19.73     0.054     312   283
Comp:  1        Halpha      64.63     0.054     312   283
Comp:  2     [OII]3726       64.1      0.15     197   304


...

Emission lines included in gas templates:
['Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[OII]3726' '[OII]3729' '[SII]6716'
 '[SII]6731' '[OIII]5007_d' '[OI]6300_d' '[NII]6583_d']
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:        -9       298     0.097     0.085
 comp.  1:       -36       261    -0.088     0.162
 comp.  2:       -63       204    -0.300    -0.300
chi2/DOF: 9498; degree = -1; mdegree = 10
method = capfit; Jac calls: 11; Func calls: 260; Status: 2
linear_method = lsqbox; Nonzero Templates (>0.1%):  24 / 161
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1        Hdelta      0.875     0.067     -36   261
Comp:  1        Hgamma      1.135     0.067     -36   261
Comp:  1         Hbeta      1.686     0.066     -36   261
Comp:  1        Halpha      33.46     0.066     -36   261
Comp:  2     [OII]3726        2.7     0.058     -63   204
C

Emission lines included in gas templates:
['Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[OII]3726' '[OII]3729' '[SII]6716'
 '[SII]6731' '[OIII]5007_d' '[OI]6300_d' '[NII]6583_d']
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       308       316    -0.018    -0.002
 comp.  1:       189       176    -0.053    -0.002
 comp.  2:       124       125    -0.049     0.101
chi2/DOF: 905.7; degree = -1; mdegree = 10
method = capfit; Jac calls: 9; Func calls: 212; Status: 2
linear_method = lsqbox; Nonzero Templates (>0.1%):  13 / 161
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1        Hdelta      1.706     0.052     189   176
Comp:  1        Hgamma      6.008     0.052     189   176
Comp:  1         Hbeta      18.18     0.051     189   176
Comp:  1        Halpha      79.88      0.05     189   176
Comp:  2     [OII]3726      24.13     0.064     124   125
C

Emission lines included in gas templates:
['Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[OII]3726' '[OII]3729' '[SII]6716'
 '[SII]6731' '[OIII]5007_d' '[OI]6300_d' '[NII]6583_d']
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:        71       304    -0.089     0.162
 comp.  1:        81       150     0.069     0.300
 comp.  2:       146       278    -0.027     0.021
chi2/DOF: 5046; degree = -1; mdegree = 10
method = capfit; Jac calls: 7; Func calls: 170; Status: 2
linear_method = lsqbox; Nonzero Templates (>0.1%):  24 / 161
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1        Hdelta    0.09576     0.056      81   150
Comp:  1        Hgamma      1.663     0.056      81   150
Comp:  1         Hbeta      4.517     0.055      81   150
Comp:  1        Halpha      32.12     0.054      81   150
Comp:  2     [OII]3726      22.04      0.13     146   278
Co

Emission lines included in gas templates:
['Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[OII]3726' '[OII]3729' '[SII]6716'
 '[SII]6731' '[OIII]5007_d' '[OI]6300_d' '[NII]6583_d']
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       724       288    -0.047     0.057
 comp.  1:      -185        42     0.300    -0.300
 comp.  2:      -109       101     0.300    -0.300
chi2/DOF: 5.684e+04; degree = -1; mdegree = 10
method = capfit; Jac calls: 9; Func calls: 212; Status: 2
linear_method = lsqbox; Nonzero Templates (>0.1%):  21 / 161
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1        Hdelta          0     0.034    -185    42
Comp:  1        Hgamma      1.819     0.033    -185    42
Comp:  1         Hbeta      7.858     0.031    -185    42
Comp:  1        Halpha      45.04     0.027    -185    42
Comp:  2     [OII]3726      4.072     0.039    -109   1

Emission lines included in gas templates:
['Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[OII]3726' '[OII]3729' '[SII]6716'
 '[SII]6731' '[OIII]5007_d' '[OI]6300_d' '[NII]6583_d']
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:      -118       307    -0.106    -0.018
 comp.  1:      -232       295    -0.023    -0.112
 comp.  2:      -210       292     0.155     0.012
chi2/DOF: 3754; degree = -1; mdegree = 10
method = capfit; Jac calls: 11; Func calls: 260; Status: 2
linear_method = lsqbox; Nonzero Templates (>0.1%):  13 / 161
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1        Hdelta      3.477      0.06    -232   295
Comp:  1        Hgamma      5.858      0.06    -232   295
Comp:  1         Hbeta      11.52      0.06    -232   295
Comp:  1        Halpha      37.87      0.06    -232   295
Comp:  2     [OII]3726          0      0.13    -210   292
C

In [5]:
zbin4_syst_vel_array = np.array(zbin4_syst_vel)

In [6]:
col0 = fits.Column(name = 'SYST_VEL', format = 'D', array = zbin4_syst_vel_array)

coldefs = fits.ColDefs([col0])
hdu = fits.BinTableHDU.from_columns(coldefs)

zbin4_sv = Table(hdu.data)

In [7]:
# (zbin4_sv).write('zbin4_sv.fits')

In [12]:
len(zbin4_syst_vel_array[zbin4_syst_vel_array < 107000])/ len(zbin4_syst_vel_array)

0.015103338632750398